In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

2. 데이터 준비
    - 손글씨 mnist

In [2]:
mnist = tf.keras.datasets.mnist

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
x_train.shape

(60000, 28, 28)

3. 채널 차원 추가

In [9]:
x_train = x_train[...,tf.newaxis]
x_test = x_test[...,tf.newaxis]

4. 데이터셋을 섞고 배치를 만듬
    - tf.data

In [15]:
train_ds = tf.data.Dataset.from_tensor_slices(
                (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

5. Keras의 모델 서브클래싱 API를 사용 tf.keras 모델 만듬

In [20]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

6. 옵티마이저와 손실함수 선택

In [21]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

7. 성능 지표 선택, 결과 출력

In [22]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')



8. tf.GradientTape를 사용하여 모델 훈련

In [29]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradientes = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradientes, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    

 9. 모델 테스트

In [30]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [31]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    
    template = '에포크: {},손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도 : {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1,손실: 0.13555318117141724, 정확도: 95.92833709716797, 테스트 손실: 0.056919123977422714, 테스트 정확도 : 98.02999877929688
에포크: 2,손실: 0.0886622965335846, 정확도: 97.31666564941406, 테스트 손실: 0.06082519516348839, 테스트 정확도 : 98.00999450683594
에포크: 3,손실: 0.0664043053984642, 정확도: 97.97611236572266, 테스트 손실: 0.05922797694802284, 테스트 정확도 : 98.1433334350586
에포크: 4,손실: 0.05328179523348808, 정확도: 98.36917114257812, 테스트 손실: 0.06177881360054016, 테스트 정확도 : 98.13999938964844
에포크: 5,손실: 0.04433905705809593, 정확도: 98.63666534423828, 테스트 손실: 0.06453446298837662, 테스트 정확도 : 98.15599822998047


In [ ]:
: